In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras import Model
from keras.layers import (
    Embedding,
    Dense,
    Dropout,
    MultiHeadAttention,
    LayerNormalization,
    Input,
    GlobalAveragePooling1D,
)
from keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from nltk.stem import ISRIStemmer
import string, re, functools, operator
import pyarabic.araby as ar
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from keras.optimizers import Adam

In [2]:
file_path = "train.xlsx"
df_train = pd.read_excel(file_path)
df_train

,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1
...,...,...
32031,التطبيق اصبح سيء للغايه نقوم بطلب لا يتم وصول ...,-1
32032,y love you,1
32033,الباقه بتخلص وبشحن مرتين باقه اضافيه ١٠٠ جنيه,-1
32034,تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجة...,-1


In [3]:
arabic_stop_words = [
    "و", "في", "من", "على", "إلى", "لا", "أو", "هو", "هي", "يكون",
    "أنا", "أنت", "هو", "هي", "نحن", "أنتم", "هم",
    "عن", "مع", "كما", "مثل", "بين", "إذا", "حتى", "منذ",
    "و", "أو", "لكن", "إذا", "إن",
    "اليوم", "غداً", "الآن", "ثم", "بعد",
    "كان", "يكون", "أصبح", "صار", "ليس", "لم",
    "هذا", "هذه", "ذلك", "تلك", 
    "كل", "على", "فيه", "منه", "عنه", "له", "به", "إليه", "لها", "فيها",
    "بها", "منها", "عنها", "إليها", "الذي", "التي", "اللذين", "اللذان", "اللتان",
    "اللتين", "هؤلاء", "ذلك", "هذه", "هذا", "تلك", "تحت", "فوق", "معه", "لديه",
    "عليه", "عليها", "أي", "هل", "إذا", "ماذا", "هناك", "هنالك", "إلى",
    "يناير", "فبراير", "مارس", "إبريل", "مايو", "يونيو", "يوليو", "أغسطس", "سبتمبر", "أكتوبر", "نوفمبر", "ديسمبر",
    "الأحد", "الاثنين", "الثلاثاء", "الأربعاء", "الخميس", "الجمعة", "السبت"
]

In [4]:
import Stemmer
st = Stemmer.Stemmer("arabic")


def text_cleaning(text, stemmer=st):
    # remove stop words and punctuation

    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    text = re.sub("(\s\d+)", "", text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text = re.sub("\d+", " ", text)
    text = ar.strip_tashkeel(text)
    text = ar.strip_tatweel(text)
    text = text.replace("#", " ")
    text = text.replace("@", " ")
    text = text.replace("_", " ")

    tokenizer = RegexpTokenizer(r"\w+")

    words = tokenizer.tokenize(text)

    stop_words = set(string.punctuation).union(set(arabic_stop_words))

    filtered_list = [word for word in words if word.casefold() not in stop_words]

    # word stemming
    stem_words = [stemmer.stemWord(word) for word in filtered_list]

    text = " ".join(map(str, stem_words))
    text = text.replace("آ", "ا")
    text = text.replace("إ", "ا")
    text = text.replace("أ", "ا")
    text = text.replace("ؤ", "و")
    text = text.replace("ئ", "ي")

    return text


df_train["new review_description"] = df_train["review_description"].apply(
    lambda text: text_cleaning(text)
)


df_train.head(20)

,review_description,rating,new review_description
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1,شرك زبال سواق تبرشم مفيش حت رقم شكاو سواق يسيب...
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1,خدم دفع طريق الك نت توقف عند اصبح فقط دفع نقد
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1,تطبيق غب جار حذف عامل اكواد خصم لما استخدم اكت...
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1,فعل تطبيق ممتاز بس لو في امكان يتيح لمستخدم تط...
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1,سيء جدا اسعار رسوم توصيل تمت واقع ب صله
5,قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...,0,قعد عشرين سنة يدور سايق بس اما توصيل اشياء جيد...
6,احلئ تطبيق,1,احلء تطبيق
7,رائع واو مدهش,1,رايع واو مدهش
8,مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...,-1,مکو بس بحری عمان غیر بس عراق مکو یعنی نجم وحد ...
9,تطبيق جميل يستاهل الخمس نجوم👍👍👍,1,تطبيق جميل استاهل خمس نجوم


In [5]:
X = df_train["new review_description"]
y = df_train["rating"].astype(int) + 1

In [6]:
tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(X)
total_words = len(tokenizer.word_index) + 1

In [7]:
sequences = tokenizer.texts_to_sequences(X)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, y, test_size=0.2, random_state=42, stratify=y
)
num_classes = 3
y_train = np.array(y_train)
y_val = np.array(y_val)
print("X_train shape :", X_train.shape)
print("y_train shape :", y_train.shape)
print("X_val shape :", X_val.shape)
print("y_val shape :", y_val.shape)
print('Total Vocabulary Size (Untrimmed): %d' % total_words)
print('max_len: %d' % max_len)

X_train shape : (31036, 318)
y_train shape : (31036,)
X_val shape : (1000, 318)
y_val shape : (1000,)
Total Vocabulary Size (Untrimmed): 22752
max_len: 318


In [15]:
embed_dim = 32
num_heads = 10
ff_dim = 16
max_len = 318  
total_words = 22752 

# Token and Position Embedding
inputs = Input(shape=(max_len,))
token_emb = Embedding(input_dim=total_words, output_dim=embed_dim)(inputs)

positions = tf.range(start=0, limit=max_len, delta=1)
pos_emb = Embedding(input_dim=max_len, output_dim=embed_dim)(positions)

x = token_emb + pos_emb

# Encoder Block
attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
attn_output = Dropout(0.5)(attn_output)
out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)

ffn_output = Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])(out1)

ffn_output = Dropout(0.5)(ffn_output)
x = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

# Global Average Pooling and Dense layers
x = GlobalAveragePooling1D()(x)
x = Dropout(0.5)(x)
x = Dense(ff_dim, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(3, activation="softmax")(
    x
)  # Adjust the output size for the number of classes

# Build the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    tf.keras.optimizers.Adam(0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Print the model summary
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 318)]        0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 318, 32)      728064      ['input_3[0][0]']                
                                                                                                  
 tf.__operators__.add_6 (TFOpLa  (None, 318, 32)     0           ['embedding_4[0][0]']            
 mbda)                                                                                            
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 318, 32)     41952       ['tf.__operators__.add_6[0]

In [16]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [18]:
checkpoint = ModelCheckpoint(
    filepath="./models_hdf5/transformer.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weight_dict,
    callbacks=[checkpoint],
)

Epoch 1/30
970/970 [==============================] - 54s 56ms/step - loss: 1.1166 - accuracy: 0.2786 - val_loss: 1.0774 - val_accuracy: 0.3540
Epoch 2/30
970/970 [==============================] - 53s 55ms/step - loss: 1.1022 - accuracy: 0.2772 - val_loss: 1.0982 - val_accuracy: 0.3530
Epoch 3/30
970/970 [==============================] - 54s 56ms/step - loss: 1.1023 - accuracy: 0.2780 - val_loss: 1.0840 - val_accuracy: 0.3540
Epoch 4/30
970/970 [==============================] - 53s 55ms/step - loss: 1.1028 - accuracy: 0.2514 - val_loss: 1.1052 - val_accuracy: 0.0470
Epoch 5/30
970/970 [==============================] - 53s 54ms/step - loss: 1.1006 - accuracy: 0.2336 - val_loss: 1.1059 - val_accuracy: 0.0470
Epoch 6/30
970/970 [==============================] - 52s 54ms/step - loss: 1.1010 - accuracy: 0.2011 - val_loss: 1.1064 - val_accuracy: 0.0470
Epoch 7/30
970/970 [==============================] - 52s 54ms/step - loss: 1.0999 - accuracy: 0.2087 - val_loss: 1.1052 - val_accuracy:

In [19]:
model.load_weights("./models_hdf5/transformer.hdf5")
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

32/32 [==============================] - 1s 23ms/step - loss: 0.6614 - accuracy: 0.7580
Validation Loss: 0.6614454388618469, Validation Accuracy: 0.7580000162124634


In [20]:
file_path = "test _no_label.csv" # Path on Kaggle: /kaggle/input/arabic-sentiment-analysis-nlp/test _no_label.csv
df_test = pd.read_csv(file_path)
df_test

,ID,review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3,4,شغل طيب
4,5,بعد ماجربت
...,...,...
995,996,يستهل
996,997,خدمة سيئة بكل المعايير
997,998,لؤي٠٣٣٢لؤ٣٤٣س
998,999,تطبيق غير صادق ف خصم الكوبونات


In [21]:
df_test["new review_description"] = df_test["review_description"].apply(
    lambda x: text_cleaning(x)
)
df_test.head(20)

,ID,review_description,new review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...,اهنء خدم عملاء محادثه مباشره ما قصرو الله يوفق...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...,ممتاز جدا ولك اتمني ان تكون بعض مسابق والجوايز...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء,محمل يقول تم ايقاف حطي عشان تسوون خطاء
3,4,شغل طيب,شغل طيب
4,5,بعد ماجربت,ماجرب
5,6,وايد زين,وايد زين
6,7,سيئ جدا جدا,سيء جدا جدا
7,8,تطبيق ضعيف لا يليق بشركه كبيره مثل رأيه واغلب ...,تطبيق ضعيف يليق بشر كبير راي واغلب وقت عند است...
8,9,٠ ٠ؤ٩,ء
9,10,السواقين بيكدبوا ومبيجوش وبيكنسلوا الرحلات مع ...,سواق يكدب مبيجوش يكنسل رحل نفس انصح اوبر والل ...


In [22]:
sequences_test = tokenizer.texts_to_sequences(df_test["new review_description"])
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_len)

predictions = model.predict(padded_sequences_test)
predicted_labels = np.argmax(predictions, axis=1) - 1

output_df = pd.DataFrame({"ID": df_test["ID"], "rating": predicted_labels})

output_df.to_csv("./results/test_results_transformer.csv", index=False)

32/32 [==============================] - 1s 20ms/step


In [23]:
output_df["review_description"] = df_test["review_description"]
output_df.sample(n=20)

,ID,rating,review_description
620,621,1,رائعه وسهله
634,635,1,راع جدا
853,854,-1,زي الزفت وفي مشكله وبعمل خلل في الجهاز ومش بير...
917,918,1,ممتاز انصحكم بل تحميله
723,724,-1,مش التطبيق المفروض للمصريين طيب ليه اللغة الإن...
52,53,0,سيا سيا
63,64,0,طلبات تاخير في كل وقت ومافي متابعة من الادارة
729,730,0,خدمة التوصيل جدا بطيئه👎
69,70,0,السعر ( اغلا) والجوده ليست كالتي نشتريها بأنفس...
193,194,1,البرنامج حلو جداً بس عايزين منتجات الرجوع للمدرسة
